## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-24-15-50-27 +0000,nyt,Iran’s Leadership Stumbles After War and Unrest,https://www.nytimes.com/2026/01/24/world/middl...
1,2026-01-24-15-47-17 +0000,nyt,Xi’s Purge of China’s Military Brings Its Top ...,https://www.nytimes.com/2026/01/24/world/asia/...
2,2026-01-24-15-43-00 +0000,wsj,Trump Threatens New Tariffs on Canada Over China,https://www.wsj.com/politics/trump-threatens-n...
3,2026-01-24-15-40-30 +0000,bbc,Trump threatens Canada with 100% tariffs over ...,https://www.bbc.com/news/articles/cy4qww3w72lo...
4,2026-01-24-15-39-05 +0000,bbc,Climber delays rope-free skyscraper challenge ...,https://www.bbc.com/news/articles/c8rm135lnneo...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_3093/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
12,trump,54
371,ice,16
7,china,13
40,russia,12
205,greenland,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
21,2026-01-24-14-21-45 +0000,cbc,Trump threatens 100% tariffs on all Canadian g...,https://www.cbc.ca/news/politics/trump-tariff-...,99
2,2026-01-24-15-43-00 +0000,wsj,Trump Threatens New Tariffs on Canada Over China,https://www.wsj.com/politics/trump-threatens-n...,96
272,2026-01-23-16-09-21 +0000,nyt,Trump Withdraws Carney’s Invitation for Canada...,https://www.nytimes.com/2026/01/23/us/politics...,96
9,2026-01-24-15-14-19 +0000,nypost,Trump threatens Canada with 100% tariff after ...,https://nypost.com/2026/01/24/us-news/trump-th...,95
11,2026-01-24-15-01-46 +0000,nypost,Trump reveals to The Post ‘on land’ drug strik...,https://nypost.com/2026/01/24/us-news/trump-re...,94


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
21,99,2026-01-24-14-21-45 +0000,cbc,Trump threatens 100% tariffs on all Canadian g...,https://www.cbc.ca/news/politics/trump-tariff-...
7,56,2026-01-24-15-33-06 +0000,bbc,Two days of peace talks end as Russia continue...,https://www.bbc.com/news/articles/cwyr6p30054o...
245,52,2026-01-23-18-10-10 +0000,nypost,DHS officials debunk viral claims ICE used 5-y...,https://nypost.com/2026/01/23/us-news/dhs-offi...
25,41,2026-01-24-14-05-43 +0000,nypost,Massive winter storm Fern’s icy blast grounds ...,https://nypost.com/2026/01/24/us-news/winter-s...
11,40,2026-01-24-15-01-46 +0000,nypost,Trump reveals to The Post ‘on land’ drug strik...,https://nypost.com/2026/01/24/us-news/trump-re...
268,38,2026-01-23-16-18-59 +0000,nypost,NYC judge paves way for delivery customers to ...,https://nypost.com/2026/01/23/business/judge-d...
137,34,2026-01-24-00-03-52 +0000,bbc,'Half of my friends were killed' - the girls r...,https://www.bbc.com/news/articles/cd9eeyz933jo...
16,33,2026-01-24-14-32-14 +0000,nypost,Death by nature? These states are where you’re...,https://nypost.com/2026/01/24/us-news/animal-a...
103,30,2026-01-24-02-54-29 +0000,nypost,House Speaker Mike Johnson suggests he was nea...,https://nypost.com/2026/01/23/us-news/house-sp...
64,27,2026-01-24-10-50-00 +0000,startribune,NFL championship game picks: NFL’s final four ...,https://www.startribune.com/nfl-championship-g...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
